In [ ]:
# !pip install flask pyngrok

In [ ]:
# !ngrok config add-authtoken YOUR_NGROK_AUTHTOKEN

In [ ]:
import threading
from flask import Flask, render_template, request, jsonify
import pandas as pd
import random
from pyngrok import ngrok

app = Flask(__name__)

In [ ]:
# 포켓몬 데이터 로드
df = pd.read_csv("pokemon.csv")
cols = ['name', 'pokedex_number', 'base_total', 'hp', 'attack', 'defense', 'sp_attack', 'sp_defense', 'speed', 'type1', 'type2', 'against_bug', 'against_dark', 'against_dragon',
       'against_electric', 'against_fairy', 'against_fight', 'against_fire','against_flying', 'against_ghost', 'against_grass', 'against_ground','against_ice',
       'against_normal', 'against_poison', 'against_psychic', 'against_rock', 'against_steel', 'against_water']

df = df[cols]
df = df.rename(columns = {'against_fight' : 'against_fighting'})
df

In [ ]:
# 급소 기대값
vital_spot_per = 1/24
vital_spot = (1 * (1 - vital_spot_per)) + (1.5 * vital_spot_per)
vital_spot = round(vital_spot, 4)
vital_spot

In [ ]:
# 랜덤수 기대값
random_num = (((85 + 87 + 89 + 90 + 92 + 94 + 96 + 98) * 7.69) + ((86 + 88 + 91 + 93 + 95 + 97 + 99) * 5.13) + (100 * 2.56)) / 100
random_num

In [ ]:
# 이미지 함수 생성
def get_pokemon_image(pokemon_id):
    return f"https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/other/official-artwork/{pokemon_id}.png"

# 배틀 로직
def battle():
    num1, num2 = random.sample(range(len(df)), 2)
    A_pokemon = df.iloc[num1]
    B_pokemon = df.iloc[num2]

    log = []
    
    A_image_url = get_pokemon_image(A_pokemon['pokedex_number'])
    B_image_url = get_pokemon_image(B_pokemon['pokedex_number'])

    log.append(f"A팀의 포켓몬은 {A_pokemon['name']}입니다.")
    log.append(f"<img src='{A_image_url}' width='200'>")
    log.append(f"B팀의 포켓몬은 {B_pokemon['name']}입니다.")
    log.append(f"<img src='{B_image_url}' width='200'>")

    
    # 속도 비교하여 선공 결정
    if A_pokemon['speed'] > B_pokemon['speed']:
        log.append(f"A팀의 {A_pokemon['name']}이 선공입니다.")
        first_pokemon, second_pokemon = A_pokemon, B_pokemon
    elif A_pokemon['speed'] < B_pokemon['speed']:
        log.append(f"B팀의 {B_pokemon['name']}이 선공입니다.")
        first_pokemon, second_pokemon = B_pokemon, A_pokemon
    else:
      print('두 포켓몬의 스피드가 같으므로 종족값을 비교합니다.')
      if A_pokemon['base_total'] > B_pokemon['base_total']:
        log.append(f"A팀의 {A_pokemon['name']}이 선공입니다.")
        first_pokemon, second_pokemon = A_pokemon, B_pokemon
      elif A_pokemon['base_total'] < B_pokemon['base_total']:
        log.append(f"B팀의 {B_pokemon['name']}이 선공입니다.")
        first_pokemon, second_pokemon = B_pokemon, A_pokemon
      else:
        log.append('두 포켓몬의 종족값도 같으므로 랜덤으로 결정합니다.')
        if random.randint(1, 2) == 1:
          log.append(f"A팀의 {A_pokemon['name']}이 선공입니다.")
          first_pokemon, second_pokemon = A_pokemon, B_pokemonn
        else:
          log.append(f"B팀의 {B_pokemon['name']}이 선공입니다.")
          ffirst_pokemon, second_pokemon = B_pokemon, A_pokemon
            
    # 공격 방식 결정
    if first_pokemon['attack'] > first_pokemon['sp_attack']:
        first_pokemon_attack = first_pokemon['attack']
        second_pokemon_defense = second_pokemon['defense']
    else:
        first_pokemon_attack = first_pokemon['sp_attack']
        second_pokemon_defense = second_pokemon['sp_defense']

    if second_pokemon['attack'] > second_pokemon['sp_attack']:
        second_pokemon_attack = second_pokemon['attack']
        first_pokemon_defense = first_pokemon['defense']
    else:
        second_pokemon_attack = second_pokemon['sp_attack']
        first_pokemon_defense = first_pokemon['sp_defense']

    # 선공 포켓몬 데미지 계산
    first_pokemon_damage_type1 = ((((((50 * 2 / 5) + 2) * 80 * first_pokemon_attack / 50) / second_pokemon_defense) + 2) * vital_spot * random_num / 100) * 1.5 * second_pokemon[f'against_{first_pokemon["type1"]}']
    if pd.notna(first_pokemon['type2']):
      first_pokemon_damage_type2 = ((((((50 * 2 / 5) + 2) * 80 * first_pokemon_attack / 50) / second_pokemon_defense) + 2) * vital_spot * random_num / 100) * 1.5 * second_pokemon[f'against_{first_pokemon["type2"]}']
    else:
      first_pokemon_damage_type2 = 0
    
    if first_pokemon_damage_type1 >= first_pokemon_damage_type2:
      first_pokemon_damage = round(first_pokemon_damage_type1)
      log.append(f"{first_pokemon['name']}은 {first_pokemon['type1']} 기술이 더 강력합니다. \n이때 데미지는 {first_pokemon_damage}입니다.")
    else:
      first_pokemon_damage = round(first_pokemon_damage_type2)
      log.append(f"{first_pokemon['name']}은 {first_pokemon['type2']} 기술이 더 강력합니다. \n이때 데미지는 {first_pokemon_damage}입니다.")
 
    # 후공 포켓몬 데미지 계산
    second_pokemon_damage_type1 = ((((((50 * 2 / 5) + 2) * 80 * second_pokemon_attack / 50) / first_pokemon_defense) + 2) * vital_spot * random_num / 100) * 1.5 * first_pokemon[f'against_{second_pokemon["type1"]}']
    if pd.notna(second_pokemon['type2']):
      second_pokemon_damage_type2 = ((((((50 * 2 / 5) + 2) * 80 * second_pokemon_attack / 50) / first_pokemon_defense) + 2) * vital_spot * random_num / 100) * 1.5 * first_pokemon[f'against_{second_pokemon["type2"]}']
    else:
      second_pokemon_damage_type2 = 0
    
    if second_pokemon_damage_type1 >= second_pokemon_damage_type2:
      second_pokemon_damage = round(second_pokemon_damage_type1)
      log.append(f"{second_pokemon['name']}은 {second_pokemon['type1']} 기술이 더 강력합니다. \n이때 데미지는 {second_pokemon_damage}입니다.")

    else:
      second_pokemon_damage = round(second_pokemon_damage_type2)
      log.append(f"{second_pokemon['name']}은 {second_pokemon['type2']} 기술이 더 강력합니다. \n이때 데미지는 {second_pokemon_damage}입니다.")
        
    # 배틀 진행
    first_pokemon_hp, second_pokemon_hp = first_pokemon['hp'], second_pokemon['hp']
    while first_pokemon_hp > 0 and second_pokemon_hp > 0:
        second_pokemon_hp -= first_pokemon_damage
        log.append(f"{first_pokemon['name']}가 공격! {second_pokemon['name']}의 남은 HP: {max(second_pokemon_hp, 0)}")

        if second_pokemon_hp <= 0:
            log.append(f"🏆 승자: {first_pokemon['name']}!")
            return jsonify({"winner": first_pokemon['name'], "log": log})

        first_pokemon_hp -= second_pokemon_damage
        log.append(f"{second_pokemon['name']}가 반격! {first_pokemon['name']}의 남은 HP: {max(first_pokemon_hp, 0)}")

        if first_pokemon_hp <= 0:
            log.append(f"🏆 승자: {second_pokemon['name']}!")
            return jsonify({"winner": second_pokemon['name'], "log": log})

@app.route('/')
def home():
    return render_template("index.html")

@app.route('/battle', methods=['POST'])
def battle_route():
    return battle()  

# ngrok을 통해 외부에 공개
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

# 백그라운드에서 Flask 서버 실행
def run_flask():
    app.run(port=5000, debug=False, use_reloader=False)

thread = threading.Thread(target=run_flask)
thread.start()

In [ ]:
# ngrok 종료할 때 실행
# ngrok.kill()